In [1]:
# import all needed packages.
import findspark
findspark.init()

import pyspark
from pyspark import *

from pyspark.mllib.util import MLUtils
from pyspark import SparkContext
from pyspark.sql import *

import numpy as np

import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

sc = SparkContext()
spark = SparkSession(sc)
sc = SparkContext.getOrCreate(SparkConf())
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('warmupproject') \
    .getOrCreate()

In [2]:
source = spark.sparkContext
#Give the source location of the file to execute
file_data = source.textFile("abcd.txt")
lines = file_data.filter(lambda x: len(x) > 0)
# removing urls form the file
pullout_url = lines.filter(lambda x: not x.startswith("URL"))
sum = pullout_url.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda x, y: x + y).map(lambda x: (x[1], x[0])).sortByKey(False)
word_count = [list(i) for i in sum.collect()]
for x in word_count:
    x[1] = x[1].replace(".", "").replace('”', "").replace(',', "").replace("“", "")
# excluding stopwords from text data
stop_words = ["A","Mr","And","Also","Make","If","His","Their","It","Two","Be","More","Was","My","Her","Said","When","Last","Including","Who","Will","Or","Present","Is","Ms","Would","Just","Can","Of","He","How","Being","Only","Our","Now","Were","Have","Had","Them","Because","She","Did","One","Did","Get","That","Some","Has","Others","Been","Are","Could","Much","No","They","Most","Not","There","We","The","What","Even","The","Its","You","New","Those","All","Made","Where","Even","Do","Aboard","About","Above","Across","After","a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are","aren","aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but","by", "can","couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don","don't",
                "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "Many","many","Mr","Mr.","time","Time","hadn't", "has", "hasn","hasn't","have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i","if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me","mightn","mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not","now","o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own","re","s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so","some",
                "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there","these","they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn","wasn't","we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why","will", "with","won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your","yours","yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm","i've","let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've","we'd","we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would", "able", "abst","accordance","Against","Along","Amid","Among","Anti","Around","As","At","Before","Behind","Below","Beneath","Beside","Besides","Between","Beyond","But","By","Concerning","Considering","Despite","Down","During","Except","Excepting","Excluding","Following","For","From","In","Inside","Into","Like","Minus","Near""Of","Off","On","Onto","Opposite","Outside","Over","Past","Per","Plus","Regarding","Round","Save","Since","Than","Through","To","Toward","Towards","Under","Underneath","Unlike","Until","Up","Upon","Versus","Via","With","Within","Without","mr","Mr.","a","Mr","and","also","make","if","his","their","it","two","be","more","was","my","her","said","when","last","including","who","will","or","present","is","ms","would","just","can","of","he","how","being","only","our","now","were","have","had","them","because","she","did","one","did","get","that","some","has","others","been","are","could","much","no","they","most","not","there","we","the","what","even","The","its","you","new","those","all","made","where","even","do","aboard","about","above","across","after","against","along","amid","among","anti","around","as","at","before","behind","below","beneath","beside","besides","between","beyond","but","by","concerning","considering","despite","down","during","except","excepting","excluding","following","for","from","in","inside","into","like","minus","near""of","off","on","onto","opposite","outside","over","past","per","plus","regarding","round","save","since","than","through","to","toward","towards","under","underneath","unlike","until","up","upon","versus","via","with","within","without"]
stop_words.extend(["—", "it’s", "don’t", "i’m"])
extra_words = [y for y in word_count if y[1].lower() not in stop_words]
# top 100 in dictionary words in the list 
cloud_dictionary = {extra_words[z][1]: extra_words[z][0] for z in range(len(extra_words)) if z < 100}

In [3]:
top_new_5 = {}
for p in lines.collect():
      if p.startswith("URL"):
            classification = p.split('/')[6]
            if classification in top_new_5:
                  top_new_5[classification] += 1
            else:
                  top_new_5[classification] = 1
from collections import Counter
categories = list(dict(Counter(top_new_5).most_common(5)).keys())
categories
current_url = ""
address = {}
for i in lines.collect():
    if i.startswith("URL") :
            current_url = i
    else:
        if current_url in address:
                  address[current_url] += i
        else:
                  address[current_url] = i
#categories of top 5 in paragraphs
categories_information = ""
for article in address:
      if any(top in article for top in categories):
            categories_information += address[article]
most_viewed_wordcount =categories_information.split(" ")
most_viewed_wordcount = [(i.lower()).replace(".", "").replace('”', "").replace(',', "").replace("“", "") for i in most_viewed_wordcount]
top5_ignore_words = [i for i in most_viewed_wordcount if i not in stop_words]
categories_frequency = {}
for i in top5_ignore_words:
    if i not in categories_frequency:
            categories_frequency[i] = 1
    else:
            categories_frequency[i] += 1
top5cattop100 = dict(Counter(top_new_5).most_common(5))

In [6]:
#which categories has more number of articles
c_wise = {}
for parts in categories:
    for r in address:
        if parts in r:
            if parts in c_wise:
                c_wise[parts] += address[r]
            else:
                c_wise[parts] = address[r]
c_wise
#top 10 words in sports-category
sports10 = c_wise['sports']
sports_section_words = sports10.split(" ")
sports_section_replace = [(s.lower()).replace(".", "").replace('”', "").replace(',', "").replace("“", "") for s in sports_section_words]
sports_section_extrawords = [q for q in sports_section_replace if q not in stop_words]
Counter(sports_section_extrawords).most_common(10)

[('game', 3589),
 ('first', 3373),
 ('team', 2426),
 ('season', 2002),
 ('players', 1869),
 ('games', 1864),
 ('three', 1608),
 ('second', 1599),
 ('play', 1473),
 ('league', 1397)]

In [7]:
#top 10 words in us-category
us10 = c_wise['us']
us_words = us10.split(" ")
us_replace = [(s.lower()).replace(".", "").replace('”', "").replace(',', "").replace("“", "") for s in us_words]
sports_section_extrawords = [q for q in us_replace if q not in stop_words]
Counter(sports_section_extrawords).most_common(10)

[('trump', 4933),
 ('people', 4759),
 ('percent', 3901),
 ('year', 3494),
 ('years', 3418),
 ('states', 3278),
 ('first', 3262),
 ('company', 3191),
 ('united', 3048),
 ('state', 3044)]

In [8]:
#top 10 words in business-category
business10 = c_wise['business']
business_words = business10.split(" ")
business_replace = [(s.lower()).replace(".", "").replace('”', "").replace(',', "").replace("“", "") for s in business_words]
sports_section_extrawords = [q for q in business_replace if q not in stop_words]
Counter(sports_section_extrawords).most_common(10)

[('percent', 2670),
 ('company', 2460),
 ('million', 1730),
 ('year', 1548),
 ('companies', 1355),
 ('billion', 1326),
 ('years', 1147),
 ('people', 1114),
 ('business', 1111),
 ('chief', 1031)]

In [9]:
#top 10 words in world-category
world10 = c_wise['world']
world_words = world10.split(" ")
world_replace = [(s.lower()).replace(".", "").replace('”', "").replace(',', "").replace("“", "") for s in world_words]
sports_section_extrawords = [q for q in world_replace if q not in stop_words]
Counter(sports_section_extrawords).most_common(10)

[('government', 2279),
 ('people', 2246),
 ('united', 2012),
 ('states', 1414),
 ('european', 1404),
 ('years', 1352),
 ('state', 1338),
 ('party', 1298),
 ('officials', 1287),
 ('president', 1280)]

In [10]:
#top 10 words in nyregion-category
nyregion10 = c_wise['nyregion']
nyregion_words = nyregion10.split(" ")
nyregion_replace = [(s.lower()).replace(".", "").replace('”', "").replace(',', "").replace("“", "") for s in nyregion_words]
sports_section_extrawords = [q for q in nyregion_replace if q not in stop_words]
Counter(sports_section_extrawords).most_common(10)

[('york', 1326),
 ('city', 1308),
 ('police', 954),
 ('years', 951),
 ('people', 930),
 ('state', 922),
 ('year', 724),
 ('school', 605),
 ('de', 577),
 ('first', 555)]